In [1]:
#!pip install git+https://github.com/VarianAPIs/PyESAPI

In [2]:
import pyesapi
import atexit
import SimpleITK as sitk
import os
import numpy as np

In [3]:
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose)

<bound method 'Dispose'>

In [4]:
#for pat_sum in app.PatientSummaries:
#    print(pat_sum.Id, pat_sum.LastName, pat_sum.FirstName)

In [90]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
fid.close()
MRN = line.strip('\n')

In [5]:
patient = app.OpenPatientById(MRN)

In [6]:
courses = [c.Id for c in patient.Courses]

In [7]:
courses

['0 QA', '1 Abdomen']

In [8]:
plan = patient.CoursesLot(courses[1]).PlanSetupsLot(0)

In [9]:
print(f'Plan Id: {plan.Id}')
print(f'Dose Per Fx: {plan.PrescribedDosePerFraction}')
print(f'Number of Fx: {plan.NumberOfFractions}')

Plan Id: 1 Abdomen
Dose Per Fx: 300.0 cGy
Number of Fx: 10


In [10]:
print(f'StructureID,TYPE,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.Id},{structure.DicomType},{structure.Volume:.2f}')

StructureID,TYPE,VOLUME
Stomach,ORGAN,174.43
SpinalCanal,ORGAN,36.80
Kidney_R,ORGAN,120.39
Kidney_L,ORGAN,202.51
Kidneys,ORGAN,322.90
CTV,,415.49
BowelSpace,ORGAN,1415.11
BODY,EXTERNAL,33092.74
Liver,ORGAN,1074.83
PTV,PTV,714.05
SpinalCanal_PRV3,ORGAN,72.05
CouchSurface,SUPPORT,2597.63
CouchInterior,SUPPORT,15016.65
3000Ring,CONTROL,285.60
xPTV-ctv,PTV,300.20
xcool,CONTROL,5.13


In [11]:
ct = plan.StructureSet.Image

In [12]:
type(ct)

VMS.TPS.Common.Model.API.Image

In [65]:
numpy_ct = ct.np_array_like().transpose()

In [79]:
numpy_ct.shape

(232, 512, 512)

In [66]:
image_handle = sitk.GetImageFromArray(numpy_ct)

In [67]:
image_handle.SetSpacing((ct.get_XRes(), ct.get_YRes(), ct.get_ZRes()))
direction = [ct.get_XDirection()[i] for i in range(3)] + [ct.get_YDirection()[i] for i in range(3)] + [ct.get_ZDirection()[i] for i in range(3)]
image_handle.SetDirection(tuple(direction))

In [68]:
origin = tuple([ct.get_Origin()[i] for i in range(3)])
image_handle.SetOrigin(origin)

In [69]:
sitk.WriteImage(image_handle, r'K:\Image.nii.gz')

In [83]:
mask = plan.StructureSet.StructuresLot('Liver').np_mask_like(plan.StructureSet.Image)

In [85]:
mask = mask.transpose()

In [86]:
mask_handle = sitk.GetImageFromArray(mask)
mask_handle.SetSpacing(image_handle.GetSpacing())
mask_handle.SetOrigin(image_handle.GetOrigin())
mask_handle.SetDirection(image_handle.GetDirection())

In [87]:
sitk.WriteImage(mask_handle, r'K:\Mask.nii.gz')

In [92]:
plan.StructureSet.StructuresLot('Liver').Volume # cc

1074.8336902904346